In [5]:
import os
import json
import pyarrow.fs
import sycamore
from pinecone import Pinecone
from sycamore.functions.tokenizer import OpenAITokenizer
from sycamore.llms import OpenAIModels, OpenAI
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.merge_elements import GreedySectionMerger
from sycamore.transforms.partition import ArynPartitioner
from sycamore.transforms.embed import OpenAIEmbedder
from sycamore.materialize_config import MaterializeSourceMode
from sycamore.utils.pdf_utils import show_pages
from sycamore.transforms.summarize_images import SummarizeImages
from sycamore.context import ExecMode
from pinecone import ServerlessSpec

ctx = sycamore.init(ExecMode.LOCAL)

In [6]:
from pinecone import Pinecone

client = Pinecone(
    api_key="pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt",
    environment="us-east-1"
)

In [7]:
index_name = "demo"

all_indexes = client.list_indexes().indexes

exists = any(index['name'] == index_name for index in all_indexes)

if exists:
  index = client.Index(index_name)
else:
  client.create_index(
      name=index_name,
      dimension=1536,
      metric="cosine"
      )
  index=client.Index(index_name)

In [8]:
index_name = "demo"

existing_indexes = client.list_indexes().names()
print("Existing indexes:", existing_indexes)

if index_name not in existing_indexes:
    client.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine"
    )
    print(f"Index '{index_name}' created.")

Existing indexes: ['demo2', 'quickstart', 'demo']


In [26]:
index = client.Index(index_name)

query_docs = ctx.read.pinecone(index_name=index_name, api_key='pcsk_4P1CDJ_3Dx22sxGN8Vq2hS7TXe2MWgD2gp1QFe4z8Qnj4g86rcK3HSA9xiPXEjxpTXrLrt')
query_docs.show(show_embedding=False)
# print(typeOf(query_docs))
#doc_ids = [doc['doc_id'] for doc in query_docs.docs]  # Assuming 'docs' is the correct attribute


sycamore.executor - INFO - 2025-01-10 04:08:03,849 - Executing source PineconeReader


INFO:sycamore.executor:Executing source PineconeReader


{'doc_id': 'aryn:c-eyz5jdoo5la9mjyhe8js0f0',
 'embedding': '<1536 floats>',
 'parent_id': 'aryn:f-5m3jn27uji6pqemwkhm7cva',
 'properties': {'image_size': [597.0, 826.0],
                '_element_index': 8.0,
                'page_number': 1.0,
                'path': '/content/Digestive_System_508.pdf',
                'score': 0.8949092626571655,
                'image_mode': 'RGB',
                '_doc_source': 'DB_QUERY'},
 'bbox': [0.6039058909696691,
          0.30837296919389207,
          0.9432667451746324,
          0.6745396839488637],
 'type': 'Image',
 'text_representation': 'Mouth Esophagus Stomach Liver Gallbladder Pancreas '
                        'Small Large intestine intestine Anus Rectum',
 'elements': [],
 'lineage_id': 'wu2lmdhfucu571ta6tn4cmp'}
{'doc_id': 'aryn:c-h74qiylknpj3dqhh8l17kmf',
 'embedding': '<1536 floats>',
 'parent_id': 'aryn:f-dwz10usa1d4lwk43lwp57ez',
 'bbox': [0.23525848388671874,
          0.9615348677201705,
          0.5640300436580883,
     

In [61]:
fetched_vectors = index.fetch(ids=["aryn:f-dwz10usa1d4lwk43lwp57ez#aryn:c-8o7yrzfqi41jq6s3xfz453x", "aryn:f-5m3jn27uji6pqemwkhm7cva#aryn:c-9roxo8jxd6hkru0ttguemxl"])
print("Fetched vectors:", fetched_vectors)

{'namespace': '', 'usage': {'read_units': 10}, 'vectors': {}}


 Import Data into SingleStore

In [17]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 48.8 MB/s eta 0:00:00


In [18]:
pip show mysql-connector-python

Name: mysql-connector-python
Version: 9.1.0
Summary: A self-contained Python driver for communicating with MySQL servers, using an API that is compliant with the Python Database API Specification v2.0 (PEP 249).
Home-page: UNKNOWN
Author: Oracle and/or its affiliates
Author-email: 
License: GNU GPLv2 (with FOSS License Exception)
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [19]:
import mysql.connector

conn = mysql.connector.connect(
    host="svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com",
    user="govardhan reddy-f9a7f",
    password="fcmdPLZezPWMlTNwRxDzKbq9G6PsbGwM",
    database="db_govardhanreddy_0d754",
    port=3333,
    connection_timeout=30
)

cursor = conn.cursor()

In [20]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS pinecone_data (
    id VARCHAR(255) PRIMARY KEY,
    vector JSON,
    metadata JSON
);
""")

In [21]:
import json

fetched_vectors = {
    "aryn:f-dwz10usa1d4lwk43lwp57ez#aryn:c-8o7yrzfqi41jq6s3xfz453x": {
        "values": [0.1, 0.2, 0.3, 0.4],
        "metadata": {"description": "Some data for the first custom ID"}
    },
    "aryn:f-5m3jn27uji6pqemwkhm7cva#aryn:c-9roxo8jxd6hkru0ttguemxl": {
        "values": [0.5, 0.6, 0.7, 0.8],
        "metadata": {"description": "Some data for the second custom ID"}
    }
}

In [22]:
data_for_db = [
    (vector_id, json.dumps(vector['values']), json.dumps(vector['metadata']))
    for vector_id, vector in fetched_vectors.items()
]

In [23]:
insert_query = """
INSERT INTO pinecone_data (id, vector, metadata)
VALUES (%s, %s, %s)
ON DUPLICATE KEY UPDATE vector=VALUES(vector), metadata=VALUES(metadata);
"""

In [24]:
cursor.executemany(insert_query, data_for_db)
conn.commit()

print(f"Inserted {cursor.rowcount} rows into SingleStore!")

Inserted 0 rows into SingleStore!


In [25]:
cursor.close()
conn.close()

Call API from SingleStore

In [26]:
import mysql.connector
import requests
import json

In [52]:
conn = mysql.connector.connect(
    host="svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com",
    user="govardhan reddy-f9a7f",
    password="fcmdPLZezPWMlTNwRxDzKbq9G6PsbGwM",
    database="db_govardhanreddy_0d754",
    port=3333,
    connection_timeout=30
)
cursor = conn.cursor()

In [53]:
cursor.execute("SELECT id, vector, metadata FROM pinecone_data LIMIT 10")
rows = cursor.fetchall()

for row in rows:
  print(row)

cursor.close()
conn.close()

('aryn:f-5m3jn27uji6pqemwkhm7cva#aryn:c-9roxo8jxd6hkru0ttguemxl', '[0.5,0.6,0.7,0.8]', '{"description":"Some data for the second custom ID"}')
('id2', '[0.5,0.6,0.7,0.8]', '{"description":"Some data for id2"}')
('aryn:f-dwz10usa1d4lwk43lwp57ez#aryn:c-8o7yrzfqi41jq6s3xfz453x', '[0.1,0.2,0.3,0.4]', '{"description":"Some data for the first custom ID"}')
('id1', '[0.1,0.2,0.3,0.4]', '{"description":"Some data for id1"}')
